#### 네이버 API Search Place 중단에 따라 카카오 API 로  실습코드 변경하였습니다 #####

- 안내 페이지: http://bit.ly/pwd_kakaoAPI_guide
- 깃헙 주소: https://github.com/Play-with-data/datasalon

# 5.3 지도시각화

###  5.3.2 데이터 준비

In [16]:
# 예제 5-24 크롤링 데이터 불러오기
import pandas as pd
raw_total = pd.read_excel('./files/1_crawling_raw.xlsx')
raw_total.head()

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [17]:
# 예제 5-25 위치정보 가져오기
location_counts = raw_total['place'].value_counts( )
location_counts

Jeju                       271
Jeju-do                    179
Jungle Book by Alice       108
Seogwipo                    66
제주도 크리스마스 박물관               59
                          ... 
전주도                          1
보목                           1
단디501                        1
아산 음봉                        1
제주 반디앤로제이 펜션 & 플라워브런치카페      1
Name: place, Length: 1028, dtype: int64

In [18]:
# 예제 5-26 등록된 위치정보별 빈도수 데이터
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.head()

,place
Jeju,271
Jeju-do,179
Jungle Book by Alice,108
Seogwipo,66
제주도 크리스마스 박물관,59


In [19]:
# 예제 5-27 위치정보 빈도수 데이터 저장하기
location_counts_df.to_excel('./files/3_location_counts.xlsx')

In [ ]:
# 예제 5-28 위치정보 종류 확인하기
locations = list( location_counts.index )
locations 

### 5.3.3 카카오 검색  API를 활용한 장소 검색

In [ ]:
# 예제 5-29 카카오 검색 API 사용 예시

import requests

searching = '합정 스타벅스'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)

headers = {
    "Authorization": "KakaoAK 여기에카카오API키입력하세요"     
    # 입력시 반드시  KakaoAK 뒤에 한 칸 띄우고 API키 값을 적어야 합니다. 
}

places = requests.get(url, headers = headers).json()['documents']
places



In [ ]:
# 예제 5-30 카카오 로컬 API를 활용한 장소 검색 함수 만들기
def find_places(searching):
    # ① 접속URL 만들기
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    # ② headers 입력하기
    headers = {
    "Authorization": "KakaoAK 여기에카카오API키입력하세요"
    }
    # ③ API 요청&정보 받기
    places = requests.get(url, headers = headers).json()['documents']
    # ④ 필요한 정보 선택하기
    place = places[0] 
    name = place['place_name']
    x=place['x']
    y=place['y']
    data = [name, x, y, searching] 

    return data


In [ ]:
# 예제 5-31 제주공항 검색 예시
data = find_places('제주공항')
data

In [ ]:
# 반복작업 진행시 진행바 표시하기위한 라이브러리 tqdm 활용하기
# ! pip install tqdm   # tqdm 라이브러리 설치하기
from tqdm.notebook import tqdm

In [ ]:
# 예제 5-32 인스타그램 위치명 위치정보 검색하기
#### 검색시 시간이 많이 걸려, 현재 작업 진행현황을 볼 수 있도록,  tqdm_notebook 추가 했습니다.

import time
locations_inform = [ ]
for location in tqdm(locations):
    try:
        data = find_places(location)       
        locations_inform.append(data) 
        time.sleep(0.5) 
    except:
        pass
locations_inform


In [ ]:
# 예제 5-33 위치정보 저장하기
locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['name_official','경도','위도','인스타위치명']
locations_inform_df.to_excel('./files/3_locations.xlsx', index=False)


In [22]:
# 예제 5-34 인스타 게시량 및 위치정보 데이터 불러오기
location_counts_df = pd.read_excel('./files/3_location_counts.xlsx', index_col = 0)
locations_inform_df = pd.read_excel('./files/3_locations.xlsx')

In [24]:
# 예제 5-35 위치 데이터 병합하기
location_data = pd.merge(locations_inform_df, location_counts_df, 
                         how = 'inner', left_on = 'name_official', right_index=True)

location_data.head()


,name_official,경도,위도,인스타위치명,place
1,서귀포잠수함,126.558616,33.239303,Seogwipo,1
308,서귀포잠수함,126.558616,33.239303,서귀포잠수함,1
3,할로비치,127.020826,37.518211,할로비치,51
4,제주에인감귤밭,126.539410,33.256610,제주에인감귤밭,48
5,제주도,126.545876,33.379777,Jeju Island,4


In [25]:
# 예제 5-36 데이터 중복 점검하기
location_data['name_official'].value_counts()

오설록티뮤지엄    3
가시아방국수     2
서귀포잠수함     2
대우정        2
칠돈가 본점     2
          ..
로타리과자점     1
곱을락        1
하례정원       1
동문로터리      1
서귀포의료원     1
Name: name_official, Length: 416, dtype: int64

In [26]:
# 예제 5-37 장소 이름 기준 병합하기
location_data = location_data.pivot_table(index = ['name_official','경도','위도'], values = 'place', aggfunc='sum')
location_data.head()

,,,place
name_official,경도,위도,
73st,126.455109,33.456989,1
가드망제,126.532954,33.500991,1
가배,126.505570,33.231541,1
가시식당,126.771438,33.353060,2
가시아방국수,126.918072,33.438605,2


In [ ]:
# 예제 5-38 병합한 데이터 저장하기
location_data.to_excel('./files/3_location_inform.xlsx')

### 5.3.6 folium을 이용한 지도 시각화 ① - 개별 표시

In [27]:
# 예제 5-39 데이터 불러오기
location_data = pd.read_excel('./files/3_location_inform.xlsx')
location_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 4 columns):
name_official    416 non-null object
경도               417 non-null float64
위도               417 non-null float64
place            417 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 13.2+ KB


In [28]:
# 예제 5-40 지도 표시하기
### 변수명 수정  수정일자 2020.02.17

import folium

Mt_Hanla =[33.362500, 126.533694]
map_jeju = folium.Map(location = Mt_Hanla, zoom_start = 11)

for i in range(len(location_data)):
    name = location_data ['name_official'][i]    # 공식명칭
    count = location_data ['place'][i]           # 게시글 개수
    size = int(count)*2
    long = float(location_data['위도'][i])      
    lat = float(location_data['경도'][i])       
    folium.CircleMarker((long,lat), radius = size, color='red', popup=name).add_to(map_jeju)
    
map_jeju

In [ ]:
# 예제 5-41 지도 저장하기
map_jeju.save('./files/3_jeju.html') 

### 4.3.7 folium을 이용한 지도 시각화 ② - 그룹으로 표시

In [29]:
# 예제 5-42 지도 표시하기(마커 집합)
### 변수명 수정  수정일자 2020.02.17

from folium.plugins import MarkerCluster

locations = []
names = []

for i in range(len(location_data)):
    data = location_data.iloc[i]  # 행 하나씩
    locations.append((float(data['위도']),float(data['경도'])))    # 위도 , 경도 순으로..
    names.append(data['name_official'])


Mt_Hanla =[33.362500, 126.533694]
map_jeju2 = folium.Map(location = Mt_Hanla, zoom_start = 11)
                       
marker_cluster = MarkerCluster(
    locations=locations, popups=names,
    name='Jeju',
    overlay=True,
    control=True,

)

marker_cluster.add_to(map_jeju2)
folium.LayerControl().add_to(map_jeju2)

map_jeju2


In [ ]:
# 예제 5-43 지도 저장하기
map_jeju2.save('./files/3_jeju_cluster.html') 